# DSC Case Study

## <center>0. Project Definition

### Steps
1. Project definition
2. Data preparation
3. Model Building
4. Model Validation
5. Model Usage

## <center> 1. Libraries and Folders

In this first step we defined the directory (with our data, IESEG library and programs) and the external libraries (that we used to run some pieces of code). A more detailed explanation on the usefulness of each library can be found in the documentation.

In [ ]:
path="C:/Users/bthomasemil/Documents/Predictive/Project"


In [ ]:
pip install imblearn

In [ ]:
#External Libraries specifically added for predictive analysis
import sys
sys.path.append(path)
import numpy                as np
import pandas               as pd
import seaborn              as sns
from pandas                 import DataFrame
from pandas                 import read_csv
from pandas                 import pivot_table
from pandas                 import Series
from pandas                 import get_dummies
from numpy                  import random
from numpy                  import where
from numpy                  import nan
from numpy                  import array
from sklearn.ensemble       import RandomForestClassifier
from sklearn.ensemble       import GradientBoostingClassifier
from sklearn.metrics        import accuracy_score
from sklearn.metrics        import roc_auc_score
from sklearn.metrics        import auc
from sklearn.metrics        import recall_score
from sklearn.cluster        import KMeans
from sklearn.tree           import DecisionTreeClassifier
from sklearn.linear_model   import LogisticRegression 
from sklearn.svm            import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors      import KNeighborsClassifier
from matplotlib             import pyplot
from scipy.stats.mstats     import winsorize #we dont used it
from scipy.stats            import pearsonr
from matplotlib             import pyplot
from ieseg                  import partition
from ieseg                  import roc
from ieseg                  import lift
from ieseg                  import cumulativeResponse
from ieseg                  import cumulativeGains
from imblearn.over_sampling import SMOTE


#External ibraries specifically added for segmentation analysis
from sklearn.cluster        import AgglomerativeClustering
from pandas                 import to_numeric
from matplotlib             import pyplot
from mpl_toolkits.mplot3d   import Axes3D 

## <center> 2. Data Preparation

#### 2.1 initialization

In [ ]:
#Paths
directory = path
inputData = directory + "/Data"
sandbox   = directory + "/Sandbox"

In [ ]:
# - DATA PREP - #
# Functions used in this notebook

# Data Inspection Tools
def explore(data):
    print(data.head())
    print(data.tail())
    print(data.info())
    print(data.columns)
    print(data.dtypes)
    print(data.shape)
    print(data.describe())

# Function to change zipcode to province 
def province(zipCode):
    if((zipCode >= 1000) & (zipCode < 1300)):
        return("Brussels")
    elif(zipCode < 1500):
        return("Walloon Brabant")
    elif(zipCode < 2000):
        return("Flemish Brabant")
    elif(zipCode < 3000):
        return("Antwerp")
    elif(zipCode < 3500):
        return("Flemish Brabant")
    elif(zipCode < 4000):
        return("Limburg")
    elif(zipCode < 5000):
        return("Liege")
    elif(zipCode < 6000):
        return("Namur")
    elif(zipCode < 6600):
        return("Hainaut")
    elif(zipCode < 7000):
        return("Luxembourg")
    elif(zipCode < 8000):
        return("Hainaut")
    elif(zipCode < 9000):
        return("West Flanders")
    elif(zipCode < 10000):
        return("East Flanders")
    else:
        return("Missing")

# Function to change zipcode to  region  
def region(zipCode):
    if((zipCode >= 1000) & (zipCode < 1300)):
          return("Brussels")
    elif(zipCode < 1500):
          return("Wallonia")
    elif(zipCode < 2000):
          return("Flanders")
    elif(zipCode < 3000):
          return("Flanders")
    elif(zipCode < 3500):
          return("Flanders")
    elif(zipCode < 4000):
          return("Flanders")
    elif(zipCode < 5000):
          return("Wallonia")
    elif(zipCode < 6000):
          return("Wallonia")
    elif(zipCode < 6600):
          return("Wallonia")
    elif(zipCode < 7000):
          return("Wallonia")
    elif(zipCode < 8000):
          return("Wallonia")
    elif(zipCode < 9000):
          return("Flanders")
    elif(zipCode < 10000):
          return("Flanders")
    else:
          return("Missing")

#### 2.2 Donor Data

In [ ]:
#Reading the data and saving it as a dataframe called donor
donor = read_csv(inputData + "/donors.csv", sep = ';')

#Call function to inspect the donor data
explore(donor)
    
#Analyse Gender Column
donor['gender'].unique()

#Replace C S U with NA
donor[donor.gender.isin(['U'])]=donor[donor.gender.isin(['U'])].replace(['U'], 'NA')

#Check if it works
(donor.gender == 'NA').sum()
donor.tail()

#Inspect observation for the language variable
donor['language'].unique()

#Integer coding for the language variable
dictionary = {'N': 0, 'F': 1}
donor = donor.replace({'language': dictionary})

#Change the column name to make it more interpretable
donor.rename(columns = {'language' : 'is_french'}, inplace = True)

#### 2.3 Gift Data

In [ ]:
#Reading the data and saving it as a dataframe called gifts
gifts = read_csv(inputData + "/gifts.csv", sep = ';')

#Inspect the data
explore(gifts)

#Analyse the column camp id
gifts['campID'] = gifts['campID'].apply(str)

#Replace 0 in camp ID to NA
gifts['campID'] = gifts['campID'].replace('0', 'NA')

#Check the replacement of NAs
(gifts.campID == 'NA').sum()

#Extracting Year and Month from the Date column
gifts['date'] = pd.to_datetime(gifts['date'])
gifts['year'] = gifts['date'].dt.year
gifts['months'] = gifts['date'].dt.month
gifts.head()

In [ ]:
# - DATA PREP - #

#Extracting the most recent date before 2013 (assuming campaign starts on 2013-01-01)
gifts_before_2013 = gifts[(gifts.date < '2012-12-17')]
gifts_date = gifts_before_2013.groupby('donorID')['date'].max()

# SLice for 1 year
# Extracting observations in which donations have been made 1 year prior to the campaign
gifts_new_1y = gifts[(gifts.date < '2012-12-17') & (gifts.date > '2011-12-17')] 

#Grouping by Donor ofr 1 year 
gifts_1_year = gifts_new_1y.groupby('donorID')['amount'].sum()

#Slice for 3 years 
# Extracting observations in which donations have been made 3 year prior to the campaign
gifts_new_3y = gifts[(gifts.date < '2012-12-17') & (gifts.date > '2009-12-17')]
gifts_3_year = gifts_new_3y.groupby('donorID')['amount'].sum()

#Slice for 5 years 
# Extracting observations in which donations have been made 5 year prior to the campaign
gifts_new_5y = gifts[(gifts.date < '2012-12-17') & (gifts.date > '2007-12-17')]
gifts_5_year = gifts_new_5y.groupby('donorID')['amount'].sum()

#Slice for 10 years 
# Extracting observations in which donations have been made 10 year prior to the campaign
gifts_new_10y = gifts[(gifts.date < '2012-12-17') & (gifts.date > '2002-12-17')]
gifts_10_year = gifts_new_10y.groupby('donorID')['amount'].sum()

#Slice for 18 years 
# Extracting observations in which donations have been made 18 year prior to the campaign
gifts_new_18y = gifts[(gifts.date < '2012-12-17') & (gifts.date > '1995-1-17')]
gifts_18_year = gifts_new_18y.groupby('donorID')['amount'].sum()


In [ ]:
# - DATA PREP - #

# Create table for with total amount donated by donor in specific time period 

#Join data of 18y with 14y
left_table = gifts_18_year
right_table = gifts_10_year
join_18_10 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )
#Join previous with 5y
left_table = join_18_10
right_table = gifts_5_year
join_18_10_5 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )
#Join previous with 3y
left_table = join_18_10_5
right_table = gifts_3_year
join_18_10_5_3 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )
#Join previous with 1y
left_table = join_18_10_5_3
right_table = gifts_1_year
join_18_10_5_3_1 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )

#Changing names of columns
join_18_10_5_3_1.columns = ['18y', '10y','5y', '3y', '1y']

# Checking final result
final_merge_gifts = join_18_10_5_3_1

In [ ]:
# - DATA PREP - #


#Transform the dataset for better interpretation

#Create pivot with every year
gifts_1 = gifts.pivot_table(index = 'donorID', columns = ['year'], values = 'amount', aggfunc = 'sum')
#Create pivot with every month
gifts_2 = gifts.pivot_table(index = 'donorID', columns = ['months'], values = 'amount', aggfunc = 'sum')
# merge the two pivots in one table
gifts_final = pd.merge(gifts_1, gifts_2, 'left', on= "donorID")
#Fill NA with 0
gifts_final = gifts_final.iloc[:, 0:].fillna(0)
#Create Column 'frequency_gifts'
gifts_final["frequency_gifts"] = ((gifts_final.iloc[:, 0:20]) > 0.0).sum(axis = 1)
#Create quarters
gifts_final['Q1'] = gifts_final.iloc[:, -13:-10].sum(axis = 1)
gifts_final['Q2'] = gifts_final.iloc[:, -10:-7].sum(axis = 1)
gifts_final['Q3'] = gifts_final.iloc[:, 26:29].sum(axis = 1)
gifts_final['Q4'] = gifts_final.iloc[:, 29:32].sum(axis = 1)
#Drop months and yearss
gifts_quarter = gifts_final.drop(gifts_final.iloc[:,0:20],axis=1)

#Create table with quarters and gifts
left_table = final_merge_gifts
right_table = gifts_quarter
gifts_final = pd.merge(left_table,
                right_table,
                how='left',
                left_on='donorID',
                right_on='donorID'
                )

#Most recent date added to the gifts data base
left_table = gifts_final
right_table = gifts_date
gifts_final_2 = pd.merge(left_table,
                right_table,
                how='left',
                left_on='donorID',
                right_on='donorID'
                )
# create data frame with only this data
date_1 = pd.to_datetime('2012-12-17')
# add column 'recency_number' to new dataframe
gifts_final_2['recency_number'] = date_1 - gifts_final_2['date'] 
# Covert to days instead of date
gifts_final_2['recency_number'] = gifts_final_2['recency_number'] / np.timedelta64(1, 'D')
#Check data
gifts_final_2.head()

#### 2.4 Campaign 2013

In [ ]:
# - DATA PREP - #

#Read the campaign 2013 table
c_2013 = read_csv(inputData + "/campaign20130411.csv", sep = ';')
explore(c_2013)

#### 2.5 Data merge

In [ ]:
# - DATA PREP - #

#Merge the gifts_1 and c_2013 dataset
left_1 = pd.merge(c_2013, gifts_final_2, 
         how = "left",
        left_on = 'donorID', right_on = 'donorID')
left_2 = pd.merge(left_1, donor, 
         how = "left",
        left_on = 'donorID', right_on = 'donorID')

#### 2.6 Final Data

In [ ]:
# - DATA PREP - #

#Create Dummies for gender, region and province
final = pd.get_dummies(left_2, columns = ['gender'], drop_first=True)


final['amount_missing'] = where(final['amount'].isnull(), 1, 0)

# Appling condition for donations higher than 35 
final['amount_missing'] = where(final['amount']>35, 1, 0)

#Fix zipcode
final['zipcode'] = final['zipcode'].str.replace("SW6 3PS", '0')
final['zipcode'] = final['zipcode'].str.replace(" ", '0')
final['zipcode'] = final['zipcode'].fillna(0)
final['zipcode'] = final['zipcode'].astype(int)

#Apply function to zipcode to add privince and regions names
final["province"] = final["zipcode"].apply(lambda row:province(row))
final["region"] = final["zipcode"].apply(lambda row:region(row))
final.fillna(0)

final = pd.get_dummies(final, columns = ['region','province'], drop_first=True)


explore(final)


In [ ]:
# - DATA PREP - #

# Drop useless columns
final_model = final.drop(final[['donorID','amount','date','zipcode']], axis = 1)
final_model.head()

# Drop useless columns again
table_model = final_model.drop(final_model.iloc[:, 5:17],axis = 1) 
table_model.tail()

# Change NA for 0 
for column in table_model.columns:
    if column in ["18y","10y","5y","3y","1y","1","2","3","4","5","6","7","8","9","10","11","12","Q1","Q2","Q3","Q4","recency_number",'frequency_gifts']:
        table_model[column][np.isnan(table_model[column])] = 0
table_model.head()

In [ ]:
# - DATA PREP - #

#Standarization

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

table_model[['18y', '10y', '5y', '3y', '1y', 'frequency_gifts',
             'Q1', 'Q2', 'Q3','Q4', 'recency_number']] = scaler.fit_transform(table_model[['18y', '10y', '5y', '3y', '1y', 
                                                                                           'frequency_gifts', 'Q1', 'Q2', 'Q3',
                                                                                           'Q4', 'recency_number']])

#print(norma_table_model.var()) 
table_model.head()


t

In [ ]:
# - DATA PREP - #
# Check if donations made are under represented 
import seaborn as sns
sns.countplot(table_model["amount_missing"], data = table_model)

In [ ]:
data= table_model 
split_strategy = [0.8,0.2]



partitions = partition(dataFrame = data, splitStrategy = split_strategy)
for (index,dataFrame) in enumerate(partitions):
    print(f"Partition {index} shape : {dataFrame.shape} ({len(dataFrame)/len(data)})")

trainingSet   = partitions[0]
validationSet = partitions[1]



#sm = SMOTE(random_state = 12, ratio = 1.0)
#x_train, y_train = sm.fit_sample(trainingSet[features], trainingSet[target])
#sns.countplot(y_train, data = trainingSet)
#col=0
#for feat in features:
#    col+=1
#     trainingSet[feat]=pd.DataFrame(data=x_train[col])

In [ ]:
features=['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3',
       'Q4', 'recency_number', 'is_french', 'gender_F', 'gender_M',
       'gender_NA', 'gender_S', 'region_Flanders',
       'region_Wallonia', 'province_Brussels', 'province_East Flanders',
       'province_Flemish Brabant', 'province_Hainaut', 'province_Liege',
       'province_Limburg', 'province_Luxembourg', 'province_Namur',
       'province_Walloon Brabant', 'province_West Flanders']
target='amount_missing'

# col=0
# for feat in features:
#     col+=1
#     trainingSet[feat]=pd.DataFrame(data=X_train[col])

In [ ]:
print(trainingSet.isnull().sum().sum())
print(np.isnan(trainingSet).sum().sum())
trainingSet.fillna(0)
#Change the NA for 0 
for column in trainingSet.columns:
    trainingSet[column][np.isnan(trainingSet[column])] = 0
trainingSet.head()
print(trainingSet.isnull().sum().sum())
print(np.isnan(trainingSet).sum().sum())


<br>
<br>
<br>
<br>
<br>


## <center>3. Segmentation


<br>
<br>

#### 3.1 Segmentation Models

In [ ]:
for col in ["10y","5y","3y","1y","Q1","Q2","Q3","Q4"]:
    table_model[col] = winsorize(array(table_model[col]), limits=[0.00, 0.01])

In [ ]:
data=table_model
affi="manhattan"
link='single'
thre=2
clus=5
"""

This function returns the cluster of the segments.
Input: Data, Affinity, Linkage, Distance
Output: a table with the columns clusters!"

"""
features=['Q3', 'frequency_gifts', '18y', 'province_Namur', 'province_Liege', 'province_Limburg', 
                    'gender_M', 'gender_NA', 'province_East Flanders', 'province_Brussels', 'province_Flemish Brabant',
                    'province_Luxembourg', '3y', 'province_Walloon Brabant', 'province_West Flanders']
#1st part
kmeans = KMeans(n_clusters = clus)
data["kmean cluster"] = kmeans.fit_predict(data[features])

# figure = pyplot.figure().gca(projection = "3d")
# for cluster in range(5):
#     figure.scatter(data["10y"][data["kmean cluster"] == cluster], 
#                    data["3y"][data["kmean cluster"] == cluster],
#                    data["recency_number"][data["kmean cluster"] == cluster],
#                    data["gender_NA"][data["kmean cluster"] == cluster],
#                    data["5y"][data["kmean cluster"] == cluster],
#                    data["is_french"][data["kmean cluster"] == cluster],
#                    data["gender_S"][data["kmean cluster"] == cluster],
#                    data["gender_C"][data["kmean cluster"] == cluster],
#                    data["1y"][data["kmean cluster"] == cluster], 
#                    data["gender_F"][data["kmean cluster"] == cluster],
#                    marker = 'o')
# pyplot.show()

In [ ]:
for i in range(0,5):
    print(data[['Q3', 'frequency_gifts', '18y', 'province_Namur', 'province_Liege', 'province_Limburg', 
                    'gender_M', 'gender_NA', 'province_East Flanders', 'province_Brussels', 'province_Flemish Brabant',
                    'province_Luxembourg', '3y', 'province_Walloon Brabant', 'province_West Flanders']][data["kmean cluster"]==i].describe())

<br>

<br>

## <center>  4. Algorithm

<br>
<br>
<br>
<br>
<br>

#### 4.1 Stepwise Regression

In [ ]:
tree         = DecisionTreeClassifier()
logistic     = LogisticRegression(solver = "lbfgs", max_iter = 500)
randomForest = RandomForestClassifier(n_estimators = 100)
boostedTree  = GradientBoostingClassifier()
svm          = SVC(gamma = "scale", probability = True)
neuralNet    = MLPClassifier()
neighbors    = KNeighborsClassifier()
models = {"tree"         :tree,
          "logistic"     :logistic,
          "randomForest" :randomForest,
          "boostedTree"  :boostedTree,
          "svm"          :svm,
          "neuralNet"    :neuralNet,
          "neighbors"    :neighbors
         }

In [ ]:
def stepwiseRegresion (model ,trainingSet, testSet, selectedFeatures: [str], target: [str]) -> DataFrame:

    def computeAUC (forFeatures: [str]) -> ([str],float,float):
        model.fit(trainingSet[forFeatures], trainingSet[target])

        trainingSet["proba donor stepwise"] = DataFrame(model.predict_proba(trainingSet[forFeatures]))[1]
        testSet["proba donor stepwise"]     = DataFrame(model.predict_proba(testSet[forFeatures]))[1]
        
        aucTraining = roc_auc_score(array(trainingSet[target]),array(trainingSet["proba donor stepwise"]))
        aucTest     = roc_auc_score(array(testSet[target]),array(testSet["proba donor stepwise"]))

        trainingSet.drop("proba donor stepwise", axis = 1)
        testSet.drop("proba donor stepwise", axis = 1)

        return (forFeatures,aucTraining,aucTest)

    featuresOrder = []
    forwardSelection = []

    for step in range(len(selectedFeatures)):
        print(f"step {step+1}")
        aucs = []
        for feature in selectedFeatures:
            if feature not in featuresOrder:
                modelFeatures = featuresOrder.copy()
                modelFeatures.append(feature)
                aucs.append(computeAUC(forFeatures = modelFeatures))

        steps = DataFrame(aucs)
        steps.columns = ["Feature","AUC training", "AUC test"]
        steps = steps.sort_values(by=["AUC test"], ascending = False)

        featuresOrder = steps["Feature"].iloc[0]
        forwardSelection.append((step+1, steps["Feature"].iloc[0],steps["AUC training"].iloc[0],steps["AUC test"].iloc[0]))

    df = DataFrame(forwardSelection)
    df.columns = ("Step","Features","AUC Train","AUC Test")
    
    return df

#### 4.1.1. GradientBoosingClassifier

In [ ]:
forwardSelection = stepwiseRegresion(model = GradientBoostingClassifier(),
                                     trainingSet      = trainingSet, 
                                     testSet          = validationSet, 
                                     selectedFeatures = ['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3','Q4', 'recency_number', 'is_french', 'gender_M', 'gender_NA', 'gender_S', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Liege','province_Limburg', 'province_Luxembourg', 'province_Namur','province_Walloon Brabant', 'province_West Flanders'],
                                     target           = 'amount_missing')
forwardSelection.sort_values("AUC Train",ascending=False).head(10)
selectedfeaturesRFC=forwardSelection.sort_values("AUC Test",ascending=False).get_value(0,1,takeable=True)
print(selectedfeaturesRFC)
print(forwardSelection.sort_values("AUC Test",ascending=False).head(10))

#### 4.1.2. SVC

In [ ]:
forwardSelection = stepwiseRegresion(model = SVC(gamma = "scale", probability = True),
                                     trainingSet      = trainingSet, 
                                     testSet          = validationSet, 
                                     selectedFeatures = ['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3','Q4', 'recency_number', 'is_french', 'gender_M', 'gender_NA', 'gender_S', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Liege','province_Limburg', 'province_Luxembourg', 'province_Namur','province_Walloon Brabant', 'province_West Flanders'],
                                     target           = 'amount_missing')
forwardSelection.sort_values("AUC Train",ascending=False).head(10)
selectedfeaturesRFC=forwardSelection.sort_values("AUC Test",ascending=False).get_value(0,1,takeable=True)
print(selectedfeaturesRFC)
print(forwardSelection.sort_values("AUC Test",ascending=False).head(10))


#### 4.1.3. Logistic Regression

In [ ]:
forwardSelection = stepwiseRegresion(model = SVC(gamma = "scale", probability = True),
                                     trainingSet      = trainingSet, 
                                     testSet          = validationSet, 
                                     selectedFeatures = ['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3','Q4', 'recency_number', 'is_french', 'gender_M', 'gender_NA', 'gender_S', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Liege','province_Limburg', 'province_Luxembourg', 'province_Namur','province_Walloon Brabant', 'province_West Flanders'],
                                     target           = 'amount_missing')
forwardSelection.sort_values("AUC Train",ascending=False).head(10)
selectedfeaturesRFC=forwardSelection.sort_values("AUC Test",ascending=False).get_value(0,1,takeable=True)
print(selectedfeaturesRFC)
print(forwardSelection.sort_values("AUC Test",ascending=False).head(10))

#### 4.1.4. KNN

In [ ]:
forwardSelection = stepwiseRegresion(model = KNeighborsClassifier(),
                                     trainingSet      = trainingSet, 
                                     testSet          = validationSet, 
                                     selectedFeatures = ['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3','Q4', 'recency_number', 'is_french', 'gender_M', 'gender_NA', 'gender_S', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Liege','province_Limburg', 'province_Luxembourg', 'province_Namur','province_Walloon Brabant', 'province_West Flanders'],
                                     target           = 'amount_missing')
forwardSelection.sort_values("AUC Train",ascending=False).head(10)
selectedfeaturesRFC=forwardSelection.sort_values("AUC Test",ascending=False).get_value(0,1,takeable=True)
print(selectedfeaturesRFC)
print(forwardSelection.sort_values("AUC Test",ascending=False).head(10))

#### 4.1.5. Decission Tree

In [ ]:
forwardSelection = stepwiseRegresion(model = DecisionTreeClassifier(),
                                     trainingSet      = trainingSet, 
                                     testSet          = validationSet, 
                                     selectedFeatures = ['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3','Q4', 'recency_number', 'is_french', 'gender_M', 'gender_NA', 'gender_S', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Liege','province_Limburg', 'province_Luxembourg', 'province_Namur','province_Walloon Brabant', 'province_West Flanders'],
                                     target           = 'amount_missing')
forwardSelection.sort_values("AUC Train",ascending=False).head(10)
selectedfeaturesRFC=forwardSelection.sort_values("AUC Test",ascending=False).get_value(0,1,takeable=True)
print(selectedfeaturesRFC)
print(forwardSelection.sort_values("AUC Test",ascending=False).head(10))

#### 4.1.6. Random Forest

In [ ]:
forwardSelection = stepwiseRegresion(model = RandomForestClassifier(n_estimators = 100),
                                     trainingSet      = trainingSet, 
                                     testSet          = validationSet, 
                                     selectedFeatures = ['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3','Q4', 'recency_number', 'is_french', 'gender_M', 'gender_NA', 'gender_S', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Liege','province_Limburg', 'province_Luxembourg', 'province_Namur','province_Walloon Brabant', 'province_West Flanders'],
                                     target           = 'amount_missing')
forwardSelection.sort_values("AUC Train",ascending=False).head(10)
selectedfeaturesRFC=forwardSelection.sort_values("AUC Test",ascending=False).get_value(0,1,takeable=True)
print(selectedfeaturesRFC)
print(forwardSelection.sort_values("AUC Test",ascending=False).head(10))

#### 4.1.7. MLP Classifier

In [ ]:
forwardSelection = stepwiseRegresion(model = MLPClassifier(),
                                     trainingSet      = trainingSet, 
                                     testSet          = validationSet, 
                                     selectedFeatures = ['18y', '10y', '5y', '3y', '1y', 'frequency_gifts', 'Q1', 'Q2', 'Q3','Q4', 'recency_number', 'is_french', 'gender_M', 'gender_NA', 'gender_S', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Liege','province_Limburg', 'province_Luxembourg', 'province_Namur','province_Walloon Brabant', 'province_West Flanders'],
                                     target           = 'amount_missing')
forwardSelection.sort_values("AUC Train",ascending=False).head(10)
selectedfeaturesRFC=forwardSelection.sort_values("AUC Test",ascending=False).get_value(0,1,takeable=True)
print(selectedfeaturesRFC)
print(forwardSelection.sort_values("AUC Test",ascending=False).head(10))

<br>
<br>
<br>
<br>
<br>
              
              
## <center>5. Test Set


#### 5.1 Initialization 

In [ ]:
# - TEST SET - #

gifts_14 = read_csv(inputData + "/gifts.csv", sep = ';')

#Inspect the data
explore(gifts_14)

#Analyse the column camp id
gifts_14['campID'] = gifts_14['campID'].apply(str)

#Replace 0 in camp ID to NA
gifts_14['campID'] = gifts_14['campID'].replace('0', 'NA')

#Check the replacement of NAs
(gifts_14.campID == 'NA').sum()

#Extracting Year from the Date column
gifts_14['date'] = pd.to_datetime(gifts_14['date'])
gifts_14['year'] = gifts_14['date'].dt.year
gifts_14['months'] = gifts_14['date'].dt.month

In [ ]:
# - TEST SET - #

#Most recent date before 2013
gifts_before_2014 = gifts_14[(gifts_14.date < '2013-12-17')]
gifts_date_14 = gifts_before_2014.groupby('donorID')['date'].max()

# Slice of 1 year
gifts_new_1y_14 = gifts_14[(gifts_14.date < '2013-12-17') & (gifts_14.date > '2012-12-17')] 

#Grouping by Donor ofr 1 year 
gifts_1_year_14 = gifts_new_1y_14.groupby('donorID')['amount'].sum()

#Slice for 3 years 
gifts_new_3y_14 = gifts_14[(gifts_14.date < '2013-12-17') & (gifts_14.date > '2010-12-17')]
gifts_3_year_14 = gifts_new_3y_14.groupby('donorID')['amount'].sum()

#Slice for 5 years 
gifts_new_5y_14 = gifts_14[(gifts_14.date < '2013-12-17') & (gifts_14.date > '2008-12-17')]
gifts_5_year_14 = gifts_new_5y_14.groupby('donorID')['amount'].sum()

#Slice for 10 years 
gifts_new_10y_14 = gifts_14[(gifts_14.date < '2013-12-17') & (gifts_14.date > '2003-12-17')]
gifts_10_year_14 = gifts_new_10y_14.groupby('donorID')['amount'].sum()

#Slice for 18 years 
gifts_new_18y_14 = gifts[(gifts.date < '2013-12-17') & (gifts_14.date > '1995-1-17')]
gifts_18_year_14 = gifts_new_18y.groupby('donorID')['amount'].sum()


In [ ]:
# - TEST SET - #

#Create table for with total amount donated by donor in specific time period 

#Join data of 18y with 14y
left_table = gifts_18_year_14
right_table = gifts_10_year_14
join_18_10_14 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )
#Join previous with 5y
left_table = join_18_10_14
right_table = gifts_5_year_14
join_18_10_5_14 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )
#Join previous with 3y
left_table = join_18_10_5_14
right_table = gifts_3_year_14
join_18_10_5_3_14 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )
#Join previous with 1y
left_table = join_18_10_5_3_14
right_table = gifts_1_year_14
join_18_10_5_3_1_14 = pd.merge(left_table,
        right_table,
        how='left',
        left_on='donorID',
        right_on='donorID'
        )
#Changing names of columns
join_18_10_5_3_1_14.columns = ['18y','10y','5y', '3y', '1y']
final_merge_gifts_14 = join_18_10_5_3_1_14

# Checking final result
final_merge_gifts_14.head()

In [ ]:
# - TEST SET - #


#Transform the dataset for better interpretation

#Create pivot with every year
gifts_1_14 = gifts_14.pivot_table(index = 'donorID', columns = ['year'], values = 'amount', aggfunc = 'sum')

#Create pivot with every month
gifts_2_14 = gifts_14.pivot_table(index = 'donorID', columns = ['months'], values = 'amount', aggfunc = 'sum')

# merge the two pivots in one table
gifts_final_14 = pd.merge(gifts_1, gifts_2, 'left', on= "donorID")

#Fill NA with 0
gifts_final_14 = gifts_final_14.iloc[:, 0:].fillna(0)

#Create Column 'frequency_gifts'
gifts_final_14["frequency_gifts"] = ((gifts_final_14.iloc[:, 0:20]) > 0.0).sum(axis = 1)

#Create quarters
gifts_final_14['Q1'] = gifts_final_14.iloc[:, -13:-10].sum(axis = 1)
gifts_final_14['Q2'] = gifts_final_14.iloc[:, -10:-7].sum(axis = 1)
gifts_final_14['Q3'] = gifts_final_14.iloc[:, 26:29].sum(axis = 1)
gifts_final_14['Q4'] = gifts_final_14.iloc[:, 29:32].sum(axis = 1)

#Check data
gifts_final_14.head()

In [ ]:
# - TEST SET - #

#Drop months and yearss
gifts_quarter_14 = gifts_final_14.drop(gifts_final_14.iloc[:,0:20], axis=1)

#Create table with quarters and gifts
left_table_a = final_merge_gifts_14
right_table_a = gifts_quarter_14

gifts_final_1_14 = pd.merge(left_table_a,
                            right_table_a,
                            how='left',
                            left_on='donorID',
                            right_on='donorID'
                            )

#Most recent date added to the gifts data base
left_table_14 = gifts_final_1_14
right_table_14 = gifts_date_14

gifts_final_2_14 = pd.merge(left_table_14,
                            right_table_14,
                            how='left',
                            left_on='donorID',
                            right_on='donorID'
                            )


# create data frame with only this data
date_1 = pd.to_datetime('2013-12-17')

# add column 'recency_number' to new dataframe
gifts_final_2_14['recency_number'] = date_1 - gifts_final_2_14['date'] 

# Covert to days instead of date
gifts_final_2_14['recency_number'] = gifts_final_2_14['recency_number'] / np.timedelta64(1, 'D')
gifts_final_2_14.head()

In [ ]:
# - TEST SET - #

#Read the campaign 2014 table
c_2014 = read_csv(inputData + "/campaign20140115.csv", sep = ';')
explore(c_2014)

In [ ]:
# - TEST SET - #

#Merge the gifts_1 and c_2013 dataset
left_1_14 = pd.merge(c_2014, 
                     gifts_final_2_14, 
                     how = "left",
                     left_on = 'donorID', 
                     right_on = 'donorID')

left_2_14 = pd.merge(left_1_14, 
                    donor, 
                    how = "left",
                    left_on = 'donorID', 
                    right_on = 'donorID')

In [ ]:
# - DATA PREP - #

#Create Dummies for gender, region and province
final = pd.get_dummies(left_2, columns = ['gender'], drop_first=True)


final['amount_missing'] = where(final['amount'].isnull(), 1, 0)

# Appling condition for donations higher than 35 
final['amount_missing'] = where(final['amount']>35, 1, 0)

#Fix zipcode
final['zipcode'] = final['zipcode'].str.replace("SW6 3PS", '0')
final['zipcode'] = final['zipcode'].str.replace(" ", '0')
final['zipcode'] = final['zipcode'].fillna(0)
final['zipcode'] = final['zipcode'].astype(int)

#Apply function to zipcode to add privince and regions names
final["province"] = final["zipcode"].apply(lambda row:province(row))
final["region"] = final["zipcode"].apply(lambda row:region(row))
final.fillna(0)

final = pd.get_dummies(final, columns = ['region'], drop_first=True)
final = pd.get_dummies(final, columns = ['province'], drop_first=True)

explore(final)


In [ ]:

final_model_14 = final.drop(final[['donorID','amount','date','zipcode']], axis = 1)
final_model_14.columns

In [ ]:
# - TEST SET - #

#We have to drop one Gender beause it is redundant. It can cause ''Multicollinearity'' issues 
#Zipcode not necesary cause we have the dummies 
# change is out cause we have recency now
# amount is our target
# donorID is not needed either

final_model_14 = final.drop(final[['donorID','amount','date','zipcode']], axis = 1)
final_model_14.head()

#Eliminate the months 
table_model_14 = final_model_14.drop(final_model_14.iloc[:, 5:17],axis = 1) 
table_model_14.tail()

#Change the NA for 0 
for column in table_model_14.columns:
    if column in ["18y","10y","5y","3y","1y","1","2","3","4","5","6","7","8","9","10","11","12","Q1","Q2","Q3","Q4","recency_number",'frequency_gifts']:
        table_model_14[column][np.isnan(table_model_14[column])] = 0
table_model_14.head()

<br>
<br>
<br>
<br>
<br>

## <center>6. Modeling the test set 

In [ ]:
#Using the Best Model, Gradient Boosting for classification

selectedFeatures = ['Q3', 'frequency_gifts', '18y', 'province_Namur', 'province_Liege', 'province_Limburg', 
                    'gender_M', 'gender_NA', 'province_East Flanders', 'province_Brussels', 'province_Flemish Brabant',
                    'province_Luxembourg', '3y', 'province_Walloon Brabant', 'province_West Flanders']
target = "amount_missing"
testSet=table_model_14

model = GradientBoostingClassifier()
model.fit(trainingSet[selectedFeatures], trainingSet[target])
trainingSet["proba donor"] = DataFrame(model.predict_proba(trainingSet[selectedFeatures]))[1]
testSet["proba donor"]     = DataFrame(model.predict_proba(testSet[selectedFeatures]))[1]
rocTraining = roc(dataSet = trainingSet, actuals = target, probability = "proba donor")
rocTest     = roc(dataSet = testSet,     actuals = target, probability = "proba donor")

predictions   = model.predict(testSet[selectedFeatures])
probabilities = DataFrame(model.predict_proba(testSet[selectedFeatures]))[1]
accuracy      = accuracy_score(testSet[target],predictions)
auc2           = roc_auc_score(array(testSet[target]),array(probabilities))

print(accuracy)
print(auc2)
print(rocTraining)

In [ ]:
#ROC training set

pyplot.plot(rocTraining["False positive rate"], rocTraining["True positive rate"])
pyplot.plot([0, 1], [0, 1], 'k--')
pyplot.xlim([0.0, 1.0])
pyplot.ylim([0.0, 1.0])
pyplot.xlabel("False Positive Rate or (1 - Specifity)")
pyplot.ylabel("True Positive Rate or (Sensitivity)")
pyplot.title("Receiver Operating Characteristic")

In [ ]:
#ROC test set

pyplot.plot(rocTest["False positive rate"], rocTest["True positive rate"])
pyplot.plot([0, 1], [0, 1], 'k--')
pyplot.xlim([0.0, 1.0])
pyplot.ylim([0.0, 1.0])
pyplot.xlabel("False Positive Rate or (1 - Specifity)")
pyplot.ylabel("True Positive Rate or (Sensitivity)")
pyplot.title("Receiver Operating Characteristic")

In [ ]:
#Calculcate AUC for the best model

aucTraining = auc(rocTraining["False positive rate"],rocTraining["True positive rate"])
aucTest     = auc(rocTest["False positive rate"],    rocTest["True positive rate"])

print(f"auc training set : {aucTraining}")
print(f"auc test set     : {aucTest}")

<br>
<br>

## <center> 7. Appendix

In [ ]:
import seaborn as sns

pts2 = pts.groupby(['proba donor']).sum()
pts2.reset_index(inplace=True)
sns.barplot(y='amount_missing', x='proba donor', data=pts2)

In [ ]:
# To stratify
from sklearn.model_selection import train_test_split 
 X_train, X_test, y_train, y_test =    train_test_split(X, y, test_size=0.3,                     random_state=21, stratify=y)

#Oversampling
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                            n_redundant=0, n_repeated=0, n_classes=3,
                            n_clusters_per_class=1,
                            weights=[0.01, 0.05, 0.94],
                            class_sep=0.8, random_state=0)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_resampled, y_resampled) # doctest : +ELLIPSIS

import numpy as np
X_hetero = np.array([['xxx', 1, 1.0], ['yyy', 2, 2.0], ['zzz', 3, 3.0]],
                    dtype=np.object)
y_hetero = np.array([0, 0, 1])
X_resampled, y_resampled = ros.fit_resample(X_hetero, y_hetero)